# Welcome to SINE: SINgle Image Editing with Text-to-Image Diffusion Models!

# Step 1: Setup required libraries and models. 
This may take a few minutes.

You may optionally enable downloads with pydrive in order to authenticate and avoid drive download limits when fetching the pre-trained model.

In [ ]:
#@title Setup

import os

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

from argparse import Namespace

import sys
import numpy as np

from PIL import Image

import torch
import torchvision.transforms as transforms


sine_dir = os.path.join("/content", 'SINE')
# install requirements
!git clone https://github.com/zhang-zx/SINE.git sine_dir

%cd sine_dir/
! pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
! pip install -e git+https://github.com/openai/CLIP.git@main#egg=clip
! pip install transformers==4.18.0 einops==0.4.1 omegaconf==2.1.1 torchmetrics==0.6.0 torch-fidelity==0.3.0 kornia==0.6
! pip install -e .


download_with_pydrive = True 
    
class Downloader(object):
    def __init__(self, use_pydrive):
        self.use_pydrive = use_pydrive

        if self.use_pydrive:
            self.authenticate()
        
    def authenticate(self):
        auth.authenticate_user()
        gauth = GoogleAuth()
        gauth.credentials = GoogleCredentials.get_application_default()
        self.drive = GoogleDrive(gauth)
    
    def download_file(self, file_id, file_dst):
        if self.use_pydrive:
            downloaded = self.drive.CreateFile({'id':file_id})
            downloaded.FetchMetadata(fetch_all=True)
            downloaded.GetContentFile(file_dst)
        else:
            !gdown --id $file_id -O $file_dst

downloader = Downloader(download_with_pydrive)

pre_trained_path = os.path.join('models', 'ldm', 'stable-diffusion-v4')
os.makedirs(pre_trained_path, exist_ok=True)
!wget -O models/ldm/stable-diffusion-v4/sd-v1-4-full-ema.ckpt https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4-full-ema.ckpt 

device = 'cuda'

# Step 2: Download the selected fine-tuned model. 

In [ ]:
finetuned_models_dir = os.path.join('./models', 'finetuned')
os.makedirs(finetuned_models_dir, exist_ok=True)

orig_image_dir = './dataset'
os.makedirs(orig_image_dir, exist_ok=True)

source_model_type = 'dog w/o patch-based fine-tuning' #@param['dog w/o patch-based fine-tuning', 'dog w/ patch-based fine-tuning', 'Girl with a peral earring', 'Monalisa', 'castle w/o patch-based fine-tuning', 'castle w/ patch-based fine-tuning']
source_model_download_path = {"dog w/o patch-based fine-tuning":   "1WNc3Sx1lkQpYKjz2QFMxlzL0BX6zIORp",
                              "dog w/ patch-based fine-tuning":    "1bxjAi6ZMfchCJPanrryPCcD-rIRX7nWD",
                              "Girl with a peral earring":    "1SrvDggKGQ1cbtBWK0Kd8hRLgQCrtcgUV",
                              "Monalisa": "1gYSEVEa7okuQcU_PoUW4g0AVbneehZqN",
                              "castle w/o patch-based fine-tuning":    "1nt7QMCP_zKX155K3ZxnN_Nzf3tTTBMZx",
                              "castle w/ patch-based fine-tuning":  "1Cmga--lY-bThvtuLcq4XkI7CVyxnpbIh"}

model_names = { "dog w/o patch-based fine-tuning":   "dog_wo_patch.ckpt",
                "dog w/ patch-based fine-tuning":    "dog_w_patch.ckpt",
                "Girl with a peral earring":    "girl.ckpt",
                "Monalisa": "monalisa.ckpt",
                "castle w/o patch-based fine-tuning":    "castle_wo_patch",
                "castle w/ patch-based fine-tuning":  "castle_w_patch"}

download_string = source_model_download_path[source_model_type]
file_name = model_names[source_model_type]

if not os.path.isfile(os.path.join(finetuned_models_dir, file_name)):
    downloader.download_file(download_string, os.path.join(finetuned_models_dir, file_name))